In [ ]:
### SBB.py program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/thural2/THURAL2/updated/record.py').read())


In [ ]:
rec_date

In [ ]:
####################### FIRST PROCESS THE GROSS VOLUME SEGMENTATION METRICS###########################################
cred_fil=spark.sql("select * from anp_cabbtdct1_final.psa_elig_amsb")
cred_sbb_filter=cred_fil.toPandas()

## change psa_month to month name
cred_sbb_filter=cred_sbb_filter.astype({"psa_month": int})
import calendar
cred_sbb_filter['psa_month'] = cred_sbb_filter['psa_month'].apply(lambda x: calendar.month_name[x]).str.lower()
cred_sbb_filter.head(10)

In [ ]:
#### subset for GROSS VOLUME SEGMENTATION METRICS AND REST OF PSA METRICS
cred_sbb_filter0=cred_sbb_filter[(cred_sbb_filter.seg_ind==1)]
cred_sbb_filter00=filterfunc(cred_sbb_filter0)

df_mif_psa=cred_sbb_filter[(cred_sbb_filter.seg_ind==0)]

## create metric name for rest of psa metrics

############################# GROSS VOLUME ###########################################
## SBB TAB: TOTAL SBB GROSS VOLUME
sbb_gross_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(gross_all_tuple)]
sbb_gross_filter=sbb_gross_filter.assign(metric_name='Total SBB Gross Volume')
sbb_gross_filter=filterfunc(sbb_gross_filter)

## SBB GROSS LOC VOLUME
gross_loc_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(gross_loc_tuple)]
gross_loc_filter=gross_loc_filter.assign(metric_name='SBB Gross LOC Volume')
gross_loc_filter=filterfunc(gross_loc_filter)

## SBB GROSS LON VOLUME
gross_lon_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(gross_lon_tuple)]
gross_lon_filter=gross_lon_filter.assign(metric_name='SBB Gross LON Volume')
gross_lon_filter=filterfunc(gross_lon_filter)

## SBB GROSS PROFESSIONAL STUDENT LOC VOLUME
gross_prof_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(gross_student_loc_tuple)]
gross_prof_filter=gross_prof_filter.assign(metric_name='Gross Professional Student LOC')
gross_prof_filter=filterfunc(gross_prof_filter)

### APPEND ALL THE DATAFRAME FILTERS
gross_all=pd.concat([sbb_gross_filter,gross_loc_filter,gross_lon_filter,gross_prof_filter],axis=0)

########################## CREDIT CARDS ##########################################

## SBB TAB: TOTAL CREDIT CARDS
sbb_cc_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_cc_tuple)]
sbb_cc_filter=sbb_cc_filter.assign(metric_name='Total Credit Cards')
sbb_cc_filter=filterfunc(sbb_cc_filter)

## SBB TAB: BUSINESS VISA
sbb_bv_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(business_visa_cc_tuple)]
sbb_bv_filter=sbb_bv_filter.assign(metric_name='Credit Cards - Business VISA')
sbb_bv_filter=filterfunc(sbb_bv_filter)

## SBB TAB: BUSINESS VISA
sbb_btv_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(business_travel_tuple)]
sbb_btv_filter=sbb_btv_filter.assign(metric_name='Credit Cards - Business Travel VISA')
sbb_btv_filter=filterfunc(sbb_btv_filter)

## SBB TAB: AEROPLAN BUSINESS
sbb_aero_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(business_aero_tuple)]
sbb_aero_filter=sbb_aero_filter.assign(metric_name='Credit Cards - Aeroplan Business')
sbb_aero_filter=filterfunc(sbb_aero_filter)

## SBB TAB: BUSINESS SELECT NO FEE
sbb_nofee_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(bus_nofee_tuple)]
sbb_nofee_filter=sbb_nofee_filter.assign(metric_name='Credit Cards - Business Select No Fee')
sbb_nofee_filter=filterfunc(sbb_nofee_filter)

## SBB TAB: BUSINESS SELECT FEE
sbb_fee_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(bus_fee_tuple)]
sbb_fee_filter=sbb_fee_filter.assign(metric_name='Credit Cards - Business Select Fee')
sbb_fee_filter=filterfunc(sbb_fee_filter)

### APPEND ALL THE DATAFRAME FILTERS
cc_all=pd.concat([sbb_cc_filter,sbb_bv_filter,sbb_btv_filter,sbb_aero_filter,
                  sbb_nofee_filter,sbb_fee_filter],axis=0)

########################## MERCHANT SOLUTIONS ##########################################

## SBB TAB: TOTAL MERCHANT SOLUTIONS
merchant_all=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_mer_tuple)]
merchant_all=merchant_all.assign(metric_name='Total Merchant Solutions')
merchant_all=filterfunc(merchant_all)

################################ CMS ###############################################

## SBB TAB: TOTAL CMS
cms_all=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_cms_tuple)]
cms_all=cms_all.assign(metric_name='Total Cash Management Services (including RDC)')
cms_all=filterfunc(cms_all)

################################ COMMERCIAL VOLUME ###################################

## SBB TAB: TOTAL COMMERCIAL VOLUME
sbb_com_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_com_tuple)]
sbb_com_filter=sbb_com_filter.assign(metric_name='ALL Total Commercial Volume')
sbb_com_filter=filterfunc(sbb_com_filter)

## SBB TAB: COMMERCIAL DEPOSIT VOLUME
sbb_dep_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(com_deposit_tuple)]
sbb_dep_filter=sbb_dep_filter.assign(metric_name='All Commercial Deposit Volume')
sbb_dep_filter=filterfunc(sbb_dep_filter)

## SBB TAB: COMMERCIAL LOAN VOLUME
sbb_loan_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(com_loan_tuple)]
sbb_loan_filter=sbb_loan_filter.assign(metric_name='All Commercial Loan Volume')
sbb_loan_filter=filterfunc(sbb_loan_filter)

### APPEND ALL THE DATAFRAME FILTERS
com_all=pd.concat([sbb_com_filter,sbb_dep_filter,sbb_loan_filter],axis=0)

################################ NEW ACCOUNT OPENINGS ###############################

## SBB TAB: TOTAL ACCOUNT OPENINGS
sbb_open_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_new_tuple)]
sbb_open_filter=sbb_open_filter.assign(metric_name='Total New SBB Chequing Accounts')
sbb_open_filter=filterfunc(sbb_open_filter)

################################ TD SECURITIES DIRECT TRADE ###########################

## SBB TAB: TOTAL TD SECURITIES DIRECT TRADE
sbb_trade_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_trade_tuple)]
sbb_trade_filter=sbb_trade_filter.assign(metric_name='TD Securities Direct Trade')
sbb_trade_filter=filterfunc(sbb_trade_filter)

################################ BCP WIDGETS ##########################################

## SBB TAB: TOTAL BCP WIDGETS
sbb_bcp_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_bcp_tuple)]
sbb_bcp_filter=sbb_bcp_filter.assign(metric_name='Business Credit Protection(BCP) Widgets')
sbb_bcp_filter=filterfunc(sbb_bcp_filter)

############################# MUR MORTGAGE VOLUME######################################

## SBB TAB: MUR MORTGAGE VOLUME
sbb_mur_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_mur_tuple)]
sbb_mur_filter=sbb_mur_filter.assign(metric_name='Total MUR Mortgage Volume')
sbb_mur_filter=filterfunc(sbb_mur_filter)

################################ WEALTH REFERRAL WIDGETS #############################

## SBB TAB: WEALTH REFERRAL WIDGETS
sbb_wealth_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_wealth_tuple)]
sbb_wealth_filter=sbb_wealth_filter.assign(metric_name='Total Wealth Referrals')
sbb_wealth_filter=filterfunc(sbb_wealth_filter)

########################################### BODP ############################################

## SBB TAB: BODP
sbb_bodp_filter=cred_sbb_filter[cred_sbb_filter.product_code.isin(total_bodp_tuple)]
sbb_bodp_filter=sbb_bodp_filter.assign(metric_name='Total BODP')
sbb_bodp_filter=filterfunc(sbb_bodp_filter)


In [ ]:
################################# COUNT WIDGETS ####################################

## CONCATENATE WIDGETS DATAFRAMES (GROSS SEG,CREDIT CARDS,MERCHANT,CMS,ACCOUNT OPENINGS, TD SECURITIES,BCP,WEALTH,BODP)
psa_wid=pd.concat([cred_sbb_filter00,cc_all,merchant_all,cms_all,sbb_open_filter,sbb_trade_filter,
                     sbb_bcp_filter,sbb_wealth_filter,sbb_bodp_filter],axis=0)

##(1) group to count all metrics
amsb_cred=psa_wid.groupby(by=['pr_acf2','employee_name','psa_month',
                                      'metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'elig_ind_final': 'sum'
                     })
amsb_cred.rename(columns = {"elig_ind_final":'count_segment'}, inplace = True)

## create Total metric for Gross Widgets (sum of Segmentation metrics)
##(a) subset the segmentation metrics
cred_sbb_filter01=psa_wid[psa_wid.metric_name.isin(gross_seg_tuple)]

cred_sbb_filter01['metric_name']='Total SBB Credit Widgets'

amsb_seg=cred_sbb_filter01.groupby(by=['pr_acf2','employee_name','psa_month',
                                      'metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'elig_ind_final': 'sum'
                     })
amsb_seg.rename(columns = {"elig_ind_final":'count_segment'}, inplace = True)

## concatenate the dataframes
amsb_all99=pd.concat([amsb_cred,amsb_seg],axis=0)


##(2) transpose data
amsb_all90=amsb_all99.pivot_table(index=['pr_acf2','employee_name','metric_name',
                                        'am_cost_center','am_cost_center_name',
                                        'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
                                        'record_date'], columns=['psa_month'],
                     values='count_segment', aggfunc='first').reset_index().fillna(0)


In [ ]:
############################# PROCESS RETAIL COMMERCIAL VOLUME FOR AMSB ################################
## read in retail file
df_com_fil=spark.sql("select * from anp_cabbtdct1_final.SCORECARD_RETAIL_FILTER where prod_nm in {}".format(tuple(total_com_tuple)))
df_com_volume=df_com_fil.toPandas()

## change psa_month to month name
df_com_volume=df_com_volume.astype({"psa_month": int})
import calendar
df_com_volume['psa_month'] = df_com_volume['psa_month'].apply(lambda x: calendar.month_name[x]).str.lower()
df_com_volume.head(10)

######################## CATEGORIZE between loan and deposit and total#############################################
com_filter=df_com_volume[df_com_volume.prod_nm.isin(total_com_tuple)]
com_filter=com_filter.assign(metric_name='ALL Total Commercial Volume')
com_filter=filterfunc(com_filter)

## RETAIL TAB: COMMERCIAL DEPOSIT VOLUME
com_dep_filter=df_com_volume[df_com_volume.prod_nm.isin(com_deposit_tuple)]
com_dep_filter=com_dep_filter.assign(metric_name='All Commercial Deposit Volume')
com_dep_filter=filterfunc(com_dep_filter)

## RETAIL TAB: COMMERCIAL LOAN VOLUME
com_loan_filter=df_com_volume[df_com_volume.prod_nm.isin(com_loan_tuple)]
com_loan_filter=com_loan_filter.assign(metric_name='All Commercial Loan Volume')
com_loan_filter=filterfunc(com_loan_filter)

### APPEND ALL THE DATAFRAME FILTERS
com_all_ret=pd.concat([com_filter,com_dep_filter,com_loan_filter],axis=0)


####################### read in buddy branch ###########################
df_bud_max=spark.sql("select branch,logon,mon from \
(select branch,logon,cost_centre,cast(SUBSTR(record_date,5,2) as int) as mon, \
row_number() over(partition by logon,cast(SUBSTR(record_date,5,2) as int) order by record_date desc) as ran \
from anp_cabbtdct1_final.buddy_branch)b where ran=1")
df2=df_bud_max.withColumn("branch_0000",format_string("%04d","branch"))
buddy_branch=df2.toPandas()

buddy_branch.head(10)

## merge COM dataframe to buddy branch file
df_retail_buddy=pd.merge(com_all_ret,buddy_branch[['branch_0000','logon']],
                   how="left",left_on=['branch_no'],right_on=['branch_0000'],indicator=True)

## filter both records
df_retail_buddy1=df_retail_buddy[(df_retail_buddy['_merge'] == 'both')]

## drop logon
df_retail_buddy1=df_retail_buddy1.drop(columns=['logon','_merge'])

## keep relevant vars
df_retail_buddy2=filterfunc(df_retail_buddy1)


In [ ]:
################################# SUM VOLUME ####################################

## CONCATENATE VOLUME DATAFRAMES (GROSS VOLUME,COMMERCIAL VOLUME (SBB AND RETAIL), MUR VOLUME) ---THIS IS FOR AMSB ONLY
## AM AND SM RETAIL COMMERCIAL VOLUME IS CALCULATED SEPARATELY
psa_vol=pd.concat([gross_all,com_all,df_retail_buddy2,sbb_mur_filter],axis=0)

## group to count
psa_vol1=psa_vol.groupby(by=['pr_acf2','employee_name','psa_month','metric_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name',
                             'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'current_amount': 'sum'
                     })
psa_vol1.rename(columns = {"current_amount":'vol'}, inplace = True)


##transpose data
psa_vol2=psa_vol1.pivot_table(index=['pr_acf2','employee_name','metric_name','am_cost_center','am_cost_center_name',
                                     'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
                                     'record_date'], columns=['psa_month'],
                     values='vol', aggfunc='first').reset_index().fillna(0)


In [ ]:
#### concatenate the widgets and volume dataframes -----------THIS IS FOR AMSB ONLY (because it contains retail com vol. for amsb)
amsb_tot=pd.concat([amsb_all90,psa_vol2],axis=0)


In [ ]:
########## CREATE ALL OF THE MONTHS
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in amsb_tot]

amsb_tot.reset_index(drop=True, inplace=True)
amsb_cred2= pd.concat([net_a[complementary], amsb_tot], axis=1, join='outer')

##delete the extra filler row
amsb_cred2= amsb_cred2.drop(labels='a', axis=0)
##convert to float
amsb_cred2 = amsb_cred2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

amsb_cred2.rename(columns = {"pr_acf2":'acf2_id'}, inplace = True)
amsb_cred2['scorecard_filter']='SBB'
amsb_cred2['level']='AMSB'

## order relevant vars for AMSB
amsb_cred6=amsb_cred2[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
## NOW create QTD and YTD variables
newvarfunc(amsb_cred6)

## replace 0 with NaN so median only counts non-zeros
amsb_cred6=amsb_cred6.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_amsb_median=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
##rename median
cred_amsb_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all3=pd.merge(amsb_cred6,cred_amsb_median,on=['metric_name','sm_cost_center'],
                   how="left")

## Q2
cred_amsb_median1=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
##rename median
cred_amsb_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all4=pd.merge(cred_amsb_all3,cred_amsb_median1,on=['metric_name','sm_cost_center'],
                   how="left")

## Q3
cred_amsb_median2=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
##rename median
cred_amsb_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all5=pd.merge(cred_amsb_all4,cred_amsb_median2,on=['metric_name','sm_cost_center'],
                   how="left")

## Q4
cred_amsb_median3=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
##rename median
cred_amsb_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all6=pd.merge(cred_amsb_all5,cred_amsb_median3,on=['metric_name','sm_cost_center'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_amsb_ytdmedian=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
cred_amsb_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all7=pd.merge(cred_amsb_all6,cred_amsb_ytdmedian,on=['metric_name','sm_cost_center'],
                   how="left")

cred_amsb_all8=cred_amsb_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]
cred_amsb_all8.info()

In [ ]:
##############################  GROUP for AM (SBB FILTER)##############################

#### concatenate all SBB volume dataframes (does not include retail com volume)
psa_vol_am=pd.concat([gross_all,com_all,sbb_mur_filter],axis=0)

## group to count (include acf2 because need to concatenate later with AMSB widgets df)
psa_vol1_am=psa_vol_am.groupby(by=['pr_acf2','employee_name','psa_month','metric_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name',
                             'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'current_amount': 'sum'
                     })
psa_vol1_am.rename(columns = {"current_amount":'vol'}, inplace = True)


##transpose data
psa_vol2_am=psa_vol1_am.pivot_table(index=['pr_acf2','employee_name','metric_name','am_cost_center','am_cost_center_name',
                                     'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
                                     'record_date'], columns=['psa_month'],
                     values='vol', aggfunc='first').reset_index().fillna(0)


### concatenate the widgets and volume (does not include retail com vol.)
psa_all_am=pd.concat([amsb_all90,psa_vol2_am],axis=0)


########## CREATE ALL OF THE MONTHS
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in psa_all_am]

psa_all_am.reset_index(drop=True, inplace=True)
psa_all_am2= pd.concat([net_a[complementary], psa_all_am], axis=1, join='outer')

##delete the extra filler row
psa_all_am2= psa_all_am2.drop(labels='a', axis=0)
##convert to float
psa_all_am2 = psa_all_am2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})


In [ ]:
################################## group by AM ##########################################
cred_am=amgrpfunc(psa_all_am2)

##include the LEVEL variable
cred_am['level']='AM'
cred_am['scorecard_filter']='SBB'
cred_am['acf2_id']=''
cred_am['employee_name']=cred_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_am)

## replace 0 with NaN so median only counts non-zeros
cred_am=cred_am.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_am_median=cred_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all3=pd.merge(cred_am,cred_am_median,on=['metric_name'],
                   how="left")

## Q2
cred_am_median1=cred_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all4=pd.merge(cred_am_all3,cred_am_median1,on=['metric_name'],
                   how="left")

## Q3
cred_am_median2=cred_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all5=pd.merge(cred_am_all4,cred_am_median2,on=['metric_name'],
                   how="left")

## Q4
cred_am_median3=cred_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all6=pd.merge(cred_am_all5,cred_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_am_ytdmedian=cred_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all7=pd.merge(cred_am_all6,cred_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_am_all8=cred_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM (SBB FILTER)##############################
cred_sm=smgrpfunc(cred_am_all8)

##include the LEVEL variable
cred_sm['level']='SM'
cred_sm['acf2_id']=''
cred_sm['scorecard_filter']='SBB'
cred_sm['employee_name']=cred_sm['sm_cost_center_name']
cred_sm['am_cost_center']=''
cred_sm['am_cost_center_name']=''
cred_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_sm)

## replace 0 with NaN so median only counts non-zeros
cred_sm=cred_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_sm_median=cred_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all3=pd.merge(cred_sm,cred_sm_median,on=['metric_name'],
                   how="left")

## Q2
cred_sm_median1=cred_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all4=pd.merge(cred_sm_all3,cred_sm_median1,on=['metric_name'],
                   how="left")

## Q3
cred_sm_median2=cred_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all5=pd.merge(cred_sm_all4,cred_sm_median2,on=['metric_name'],
                   how="left")

## Q4
cred_sm_median3=cred_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all6=pd.merge(cred_sm_all5,cred_sm_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_sm_ytdmedian=cred_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all7=pd.merge(cred_sm_all6,cred_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_sm_all8=cred_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
#### concatenate AMSB, AM, SM
df_sbb_all=pd.concat([cred_amsb_all8,cred_am_all8,cred_sm_all8],axis=0)


In [ ]:
df_sbb_all.metric_name.unique()

In [ ]:
#spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
## overwrite
#df_sbb_all.write.mode("overwrite").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_SBB")

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(df_sbb_all).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_SBB") 

In [ ]:
## END OF PROGRAM